In [1]:
import asyncio
import os

from dotenv import load_dotenv

from app.api.chatwoot import ChatwootHandler

load_dotenv()

# Initialize the handler
handler = ChatwootHandler(
    api_url=os.getenv("CHATWOOT_API_URL") or "",
    api_key=os.getenv("CHATWOOT_API_KEY") or "",
    account_id=os.getenv("CHATWOOT_ACCOUNT_ID") or "",
)



In [ ]:
async def test_chatwoot():
    # Test sending a message
    conversation_id = 20  # Replace with actual conversation ID
    test_results = {}

    async def run_test(name: str, coro):
        try:
            result = await coro
            test_results[name] = {"status": "success", "result": result}
            print(f"✅ {name} succeeded:", result)
        except Exception as e:
            test_results[name] = {"status": "failed", "error": str(e)}
            print(f"❌ {name} failed:", e)

    # Test all methods
    await run_test(
        "send_message",
        handler.send_message(conversation_id=conversation_id, message="Hello, this is a test message!", private=False),
    )

    await run_test(
        "update_conversation_status",
        handler.update_conversation_status(conversation_id=conversation_id, status="pending"),
    )

    await run_test(
        "add_labels", handler.add_labels(conversation_id=conversation_id, labels=["some_label"])
    )

    await run_test("get_conversation_data", handler.get_conversation_data(conversation_id))

    await run_test(
        "assign_conversation",
        handler.assign_conversation(
            conversation_id=conversation_id,
            assignee_id=1,  # Replace with actual agent ID
        ),
    )

    await run_test(
        "update_custom_attributes",
        handler.update_custom_attributes(
            conversation_id=conversation_id, custom_attributes={"region1" : "RegionTest"}
        ),
    )

    await run_test("toggle_priority", handler.toggle_priority(conversation_id=conversation_id, priority="high"))

    # run test where we set priority to garbo value not in literal
    await run_test("toggle_priority", handler.toggle_priority(conversation_id=conversation_id, priority="garbo"))

    # Print summary
    print("\n=== Test Summary ===")
    total_tests = len(test_results)
    passed_tests = sum(1 for result in test_results.values() if result["status"] == "success")
    failed_tests = total_tests - passed_tests

    print(f"Total tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {failed_tests}")

    if failed_tests > 0:
        print("\nFailed tests:")
        for name, result in test_results.items():
            if result["status"] == "failed":
                print(f"- {name}: {result['error']}")


# Run the tests
await test_chatwoot()

In [ ]:
await handler.toggle_status(24,"open")